In [21]:
#importing the required library
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [22]:
def cars24_search(search_term,location):
    # Generate a URL for search term with location
    
    LOC = {'Delhi': '110001', 'Noida': '201301', 'Gurgoan': '122001', 'Mumbai' : '400001', 'Pune': '411001', 
           'Banglore': '560001','Hyderabad': '500001', 'Chennai': '600001', 'Kolkata': '700001', 'Ahmedabad': '380001' }
    
    search_term = search_term.replace(' ','%20')
    template = f'https://www.cars24.com/buy-used-car?sort=P&search={search_term}&storeCityId=8597&pinId={LOC[location]}'
    
    return template

In [24]:
def cars24_scrapping():
    
    # Function to scrap datas from cars 24.
    
    
      
    print("SCRAPPING THE REQUIRED DETAILS")
    driver=webdriver.Chrome(r'C:\Users\Neeraj Kumar\Downloads\chromedriver_win32 (2)\chromedriver')
    driver.maximize_window()
    
    LOC = {'Delhi': '110001', 'Noida': '201301', 'Gurgoan': '122001', 'Mumbai' : '400001', 'Pune': '411001', 
           'Banglore': '560001','Hyderabad': '500001', 'Chennai': '600001', 'Kolkata': '700001', 'Ahmedabad': '380001' }
    
    location = [keys for keys in LOC.keys()]
    
    CARS= ['SUV', 'Sedan', 'Hatchback', 'Luxury Sedan', 'Luxury SUV']
    
    URL = [cars24_search(j,i) for i in location for j in CARS]
    
    LOCATION = []
    MNF_YEAR = []
    BRAND = []
    MODEL = []
    VARIANT = []
    driven_KM = []
    NOOF_owners = []
    fuel_type = []
    PRIZE = []
        
    for i in URL:
        driver.get(i)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        time.sleep(10)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'class' : '_1l4fi'})

        description = []
        description1 = []
        description2 = []
        price = []
    
        for i in results:
            description.append(i.find('h2', {'class': "_3FpCg"}).text)
            description1.append(i.find('p', {'class': "cvakB"}).text)
            description2.append(i.find('ul', {'class': "bVR0c"},'li').text)
            price.append(i.find('div', {'class': "_7udZZ"}).text)
            LOCATION.append(driver.find_element_by_xpath("//p[@class='_1OR7d']/label").text)
    
    
        year = [i.split(' ',2)[0] for i in description]
        brand = [i.split(' ',2)[1] for i in description]
        D1= [i.split(' ',2)[-1] for i in description]

        for i in price:
            PRIZE.append(i)

        for i in year:
            MNF_YEAR.append(i)
        for i in brand:
            BRAND.append(i)
        D2 = [i.rsplit(' ',1)[0] for i in description1]
        variant = [i.split(' ') [-1]for i in description1]
        for i in variant:
            VARIANT.append(i)
    
        D3 = [D1[i]+' '+D2[i] for i in range(0,len(D1))]
    
        for i in D3:
            MODEL.append(i)
    
        dummy1 = [i.split('km')[-1]for i in description2]
        kms = [i.split('km')[0] for i in description2]
        kms = [i+'km' for i in kms]

        owners = [i.split('Owner')[0] for i in dummy1]
        owners = [i+'Owner' for i in owners]
        fuel = [i.split('Owner')[-1] for i in dummy1]

        for i in kms:
            driven_KM.append(i)

        for i in owners:
            NOOF_owners.append(i)

        for i in fuel:
            fuel_type.append(i)

    driver.close()
    used_cars = pd.DataFrame({"LOCATION" : LOCATION,
                              "MNF_YEAR" : MNF_YEAR,
                              "BRAND" : BRAND,
                              "MODEL" : MODEL,
                              "VARIANT" : VARIANT,
                              "DRIVEN_KM" : driven_KM,
                              "FUELTYPE" : fuel_type,
                              "NOOF_OWNERS" : NOOF_owners,
                              "PRICE" : PRIZE
                             })
    
    used_cars.to_csv("used_cars.csv",index=False) #Creating CSV
    print("SCRAPPED DETAILS ARE STORED AS 'used_cars.csv'")
    
    return used_cars

In [25]:
user_cars = cars24_scrapping()

SCRAPPING THE REQUIRED DETAILS


<ipython-input-24-c0ed39b15265>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(r'C:\Users\Neeraj Kumar\Downloads\chromedriver_win32 (2)\chromedriver')
<ipython-input-24-c0ed39b15265>:49: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  LOCATION.append(driver.find_element_by_xpath("//p[@class='_1OR7d']/label").text)


SCRAPPED DETAILS ARE STORED AS 'used_cars.csv'


In [26]:
user_cars

,LOCATION,MNF_YEAR,BRAND,MODEL,VARIANT,DRIVEN_KM,FUELTYPE,NOOF_OWNERS,PRICE
0,New Delhi,2017,Mahindra,TUV300 T8,Manual,"19,065 km",Diesel,1st Owner,"₹6,95,099"
1,New Delhi,2018,Maruti,Vitara Brezza VDI OPT,Manual,"50,742 km",Diesel,1st Owner,"₹7,16,999"
2,New Delhi,2016,Hyundai,Creta 1.6 SX CRDI,,"1,99,598 km",Diesel,1st Owner,"₹7,25,899"
3,New Delhi,2016,Hyundai,Creta 1.6 SX (O) CRDI,,"53,957 km",Diesel,1st Owner,"₹8,94,399"
4,New Delhi,2015,Maruti,S Cross ALPHA 1.3,Manual,"94,544 km",Diesel,1st Owner,"₹5,32,899"
...,...,...,...,...,...,...,...,...,...
7033,Ahmedabad,2019,Maruti,Eeco 5 STR WITH AC PLUSHTR,Manual,"2,475 km",Petrol,1st Owner,"₹4,61,999"
7034,Ahmedabad,2017,Maruti,Eeco 5 STR WITH AC PLUSHTR,Manual,"21,029 km",Petrol,1st Owner,"₹4,22,299"
7035,Ahmedabad,2017,Maruti,Ertiga VDI SHVS,Manual,"1,07,488 km",Diesel,2nd Owner,"₹7,19,099"
7036,Ahmedabad,2020,Maruti,Eeco 7 STR,Manual,"1,777 km",Petrol,1st Owner,"₹4,55,999"
